# AI Manga Upscale Colab *by Spladenly*

<br>Upscale black-and-white and color manga pages by 4x using AI.
<br><br>The following projects were used: ESRGAN by joeyballentine and AI models: 4x_eula_digimanga_bw_v2_nc1_307k by end user license agreement#9756 and 4x-AnimeSharp by Kim2091, both licensed under [CC-BY-NC-SA-4.0](https://creativecommons.org/licenses/by-nc-sa/4.0).

<br>

---

<br>

> When running the script, you will need to grant access to your Google Drive. This permission will be used exclusively to access the folder: "/ESRGAN". No other personal folders or files will be used.

<br>

Folder structure on Google Drive:

/ESRGAN/input (black-and-white manga pages go here) *
<br>/ESRGAN/input-color (color manga pages go here) *
<br>/ESRGAN/output (upscaled black-and-white manga pages will appear here)
<br>/ESRGAN/output-color (upscaled color manga pages will appear here)
<br><br>Manga pages can also be uploaded to subfolders.
<br><br> * Supported files: bmp, dib, jpeg, jpg, jpe, jp2, png, webp, pbm, pgm, ppm, pxm, pnm, pfm, sr, ras, tiff, tif, exr, hdr, pic, gif, tga.

> Google Drive has a delay. If you've just uploaded or deleted files -- Google Colab may not see the changes for up to 1 minute. It works the same way in the other direction: if processing has just finished -- you won't see new files on Google Drive right away. Please wait.

<br>To begin processing -- upload manga pages to previously mentioned folder(s), and click the run button bellow

In [ ]:
# @title Upscale Manga { display-mode: "form" }

import os
import torch
from google.colab import drive
from pathlib import Path

def init_dirs():
  Path("/content/gdrive/MyDrive/ESRGAN").mkdir \
   (parents=True, exist_ok=True)
  Path("/content/gdrive/MyDrive/ESRGAN/input").mkdir \
   (parents=True, exist_ok=True)
  Path("/content/gdrive/MyDrive/ESRGAN/output").mkdir \
   (parents=True, exist_ok=True)
  Path("/content/gdrive/MyDrive/ESRGAN/input-color").mkdir \
   (parents=True, exist_ok=True)
  Path("/content/gdrive/MyDrive/ESRGAN/output-color").mkdir \
   (parents=True, exist_ok=True)

def dir_contains_files(path):
  for root, dirs, files in os.walk(path):
    if files:
      return True
  return False

def ai_process_bw():
  !python ESRGAN/upscale.py -se -i /content/gdrive/MyDrive/ESRGAN/input \
  -o /content/gdrive/MyDrive/ESRGAN/output \
  ESRGAN/models/4x_eula_digimanga_bw_v2_nc1_307k.pth

def ai_process_color():
  !python ESRGAN/upscale.py -se -i /content/gdrive/MyDrive/ESRGAN/input-color \
  -o /content/gdrive/MyDrive/ESRGAN/output-color \
  ESRGAN/models/4x-AnimeSharp.pth

def main():
  if not torch.cuda.is_available():
    print("This session doesn't have a GPU.\n To connect a GPU, click:\n'Edit' -> 'Notebook settings' -> 'Hardware accelerator' = GPU; 'GPU type' = T4.\nAfter that, run this script again.")
    return

  if not os.path.exists('/content/gdrive/MyDrive'):
    print("Google Drive connection in progress...")
    drive.mount('/content/gdrive')

  if not os.path.exists('ESRGAN'):
    !git clone https://github.com/Spladenly/ESRGAN

  init_dirs()

  status = 0

  if dir_contains_files('/content/gdrive/MyDrive/ESRGAN/input'):
    print("Starting black-and-white upscale subprocess...")
    status += 1
    ai_process_bw()

  if dir_contains_files('/content/gdrive/MyDrive/ESRGAN/input-color'):
    print("Starting color upscale subprocess...")
    status += 1
    ai_process_color()

  if status == 0:
    print("No pages were found in the following directories: '/ESRGAN/input' and '/ESRGAN/input-color' on your Google Drive.\nPlease upload manga pages there, and run this script again.")
  else:
    print("The processing has been finished. The result can be downloaded from '/ESRGAN/output' and/or '/ESRGAN/output-color' on your Google Drive.\nTo process additional pages, run this script again. If you don't plan to process additional pages in the near future, please close the current session.\nThis can be done by clicking on: the inverted triangle (next to the 'RAM' and 'Disk' labels in the upper right corner) -> 'Disable and remove runtime'.\nThis will unlock the resources reserved for this session for other users.")

if __name__ == '__main__':
  main()

You may also upload CBZs and ZIPs instead of images. Use the following script to extract them before running Upscale Manga. After the extraction, the original files will be moved to recycle bin.

In [ ]:
# @title Extract { display-mode: "form" }

import os
import shutil
import zipfile
from google.colab import drive

def unpack(input_dir):
  zip_files = [f for f in os.listdir(input_dir) if f.endswith('.zip') or \
               f.endswith('.cbz')]

  print(f"Found {len(zip_files)} file(s).")
  print("Extracting...")

  for zip_file in zip_files:
      print(f'Processing {zip_file}...')

      zip_path = os.path.join(input_dir, zip_file)

      if not os.path.exists(os.path.splitext(zip_path)[0]):
        extract_folder = os.path.join(input_dir, \
                                        os.path.splitext(zip_file)[0])
        os.makedirs(extract_folder, exist_ok=True)

        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(extract_folder)

        os.remove(zip_path)
      else:
         print("The folder already exists -- skipping.")

def main():
  if not os.path.exists('/content/gdrive/MyDrive'):
    print("Google Drive connection in progress...")
    drive.mount('/content/gdrive')

  input_dir = '/content/gdrive/MyDrive/ESRGAN/input'
  input_color_dir = '/content/gdrive/MyDrive/ESRGAN/input-color'
  status = 0

  if os.path.exists(input_dir):
    print(f"Current folder: {input_dir}")
    status += 1
    unpack(input_dir)

  if os.path.exists(input_color_dir):
    print(f"Current folder: {input_color_dir}")
    status += 1
    unpack(input_color_dir)

  if status == 0:
    print("Both input and input-color directories don't exist.")

  print("Done.")

if __name__ == '__main__':
  main()